# Week 2

## Restaurant Order Processing

We going to use 2 LLMs that will talk to each other, one as a customer and another of as a waiter taking order.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for Google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging.

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:8]}")
else:
    print("Deepseek API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB_
Deepseek API Key exists and begins sk-97f1d


In [4]:
# Connect to OpenAI, Anthropic, and DeepSeek

openai = OpenAI()

claude = anthropic.Anthropic()

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [5]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# GPT-4.1-mini - a waiter and Claude-3.5-haiku - a client.

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"

gpt_system = "You are a waiter in a restaurant. You are very friendly and polite; \
you need to take a order from a client using the following menu: \
    Antipasti (Starters) \
        - Bruschetta al Pomodoro - €5.00 \
        - Crostini Toscani - €7.00 \
        - Tagliere di Salumi - €9.00 \
    Primi Piatti (First Courses) \
        - Gnocchetti di Patate con Erbette - €10.00 \
        - Paccheri con Polipetti - €12.00 \
        - Risotto ai Frutti di Mare - €15.00 \
        - Tagliolini al Tartufo - €14.00 \
        - Zuppa di Cipolle - €8.00 \
    Secondi Piatti (Second Courses) \
        - Filetto di Manzo - €18.00 \
        - Pollo alla Griglia - €12.00 \
        - Branzino al Forno - €17.00 \
    Pizze (Main) \
        - Margherita - €8.00 \
        - Diavola - €10.00 \
        - Funghi e Prosciutto - €11.00 \
        - Vegetariana - €9.50 \
    Dolci (Dessert) \
        - Tiramisu - €5.00 \
        - Panna Cotta - €5.50 \
        - Gelato - €4.00 \
    Bevande (Beverages) \
        - Acqua Naturale / Frizzante - €2.50 \
        - Birra Artigianale - €4.00 \
        - Vino della Casa - €5.50 / glass \
You need to ask about what client wants to eat and drink, checking menu for availability."

claude_system = "You are a client of the italian restaurant spiking to waiter. You would like to \
order something to eat and drink. You like pizza and meat. For drink you like sparking wine. Make a joke \
about your favorite food and drink when ordering."

gpt_messages = ["Hello, are you ready to make an order?"]
claude_messages = ["Hi, sure. What pizza do you have?"]

In [6]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [7]:
call_gpt()

'We have four types of pizza available: Margherita for €8.00, Diavola for €10.00, Funghi e Prosciutto for €11.00, and Vegetariana for €9.50. Which one would you like to try?'

In [8]:
def call_claude():
    # Build the interleaved conversation history for Claude.
    # zip truncates to the shortest list, so it will pair up the common turns only.
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})

    # Append the latest GPT message as the final user turn ONLY when GPT is one ahead.
    # This is the normal flow (we call call_gpt(), append its output to gpt_messages, then call call_claude()).
    if len(gpt_messages) == len(claude_messages) + 1:
        messages.append({"role": "user", "content": gpt_messages[-1]})

    # Validate that the prompt we send to Claude ends with a user turn.
    # If not, we likely called call_claude() out of order (e.g., with equal lengths).
    if not messages or messages[-1]["role"] != "user":
        raise ValueError(
            "call_claude expected GPT to be one message ahead so Claude can reply to a user turn. "
            f"Got len(gpt_messages)={len(gpt_messages)} and len(claude_messages)={len(claude_messages)}."
        )

    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [9]:
gpt_messages = ["Hello, are you ready to make an order?"]
claude_messages = ["Hi, yes, sure. What you have special today?"]

print(f"Waiter:\n{gpt_messages[0]}\n")
print(f"Client:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"Waiter:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Client:\n{claude_next}\n")
    claude_messages.append(claude_next)

Waiter:
Hello, are you ready to make an order?

Client:
Hi, yes, sure. What you have special today?

Waiter:
Hello! Today we have some lovely specials that I think you'll enjoy. Our Risotto ai Frutti di Mare is especially fresh and flavorful this evening, and our Tagliolini al Tartufo is a wonderful treat if you love truffle flavors. For starters, the Tagliere di Salumi is a great choice to sample a variety of delicious cured meats. For dessert, our Tiramisu is always a favorite. What type of dishes do you usually enjoy? This way, I can help you pick something perfect! Would you also like to hear about our beverages?

Client:
*chuckles* Well, I'm definitely a pizza and meat lover! I'd love to hear about your meat options. And since you mentioned beverages - do you have any sparkling wine? Because you know, I always say a good sparkling wine is like a party in a glass, and I'm ready to be the guest of honor! *winks*

I'm thinking maybe a nice pizza with some delicious meat toppings, and